In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrameCollection
from awsglue.dynamicframe import DynamicFrame

glueContext = GlueContext(SparkContext.getOrCreate())


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1623861101193_0009,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
DataSource0 = glueContext.create_dynamic_frame.from_options(
    format_options = {"jsonPath":"","multiline":False}, 
    connection_type = "s3", 
    format = "json", 
    connection_options = {"paths": ["s3://glueworkshop-ray-us-east-2/output/lab6/custom/temp1/"], "recurse":True}, 
    transformation_ctx = "DataSource0")

sparkDF = DataSource0.toDF()
sparkDF.createOrReplaceTempView("inputTable")
sparkDF.printSchema()
sparkDF.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- positiveIncrease: double (nullable = true)
 |-- totalTestResultsIncrease: double (nullable = true)

+--------+-----+----------------+------------------------+
|    date|state|positiveIncrease|totalTestResultsIncrease|
+--------+-----+----------------+------------------------+
|20210307|   AK|             0.0|                     0.0|
|20210307|   AL|           408.0|                  2347.0|
|20210307|   AR|           165.0|                  3380.0|
|20210307|   AS|             0.0|                     0.0|
|20210307|   AZ|          1335.0|                 45110.0|
|20210307|   CA|          3816.0|                133186.0|
|20210307|   CO|           840.0|                 38163.0|
|20210307|   CT|             0.0|                     0.0|
|20210307|   DC|           146.0|                  5726.0|
|20210307|   DE|           215.0|                  5867.0|
+--------+-----+----------------+-----------------

In [3]:
df = spark.sql("select TO_DATE(CAST(UNIX_TIMESTAMP(date, 'yyyyMMdd') AS TIMESTAMP)) as date, \
                       state , \
                       positiveIncrease ,  \
                       totalTestResultsIncrease \
                from   inputTable")
df.printSchema()
df.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- positiveIncrease: double (nullable = true)
 |-- totalTestResultsIncrease: double (nullable = true)

+----------+-----+----------------+------------------------+
|      date|state|positiveIncrease|totalTestResultsIncrease|
+----------+-----+----------------+------------------------+
|2021-03-07|   AK|             0.0|                     0.0|
|2021-03-07|   AL|           408.0|                  2347.0|
|2021-03-07|   AR|           165.0|                  3380.0|
|2021-03-07|   AS|             0.0|                     0.0|
|2021-03-07|   AZ|          1335.0|                 45110.0|
|2021-03-07|   CA|          3816.0|                133186.0|
|2021-03-07|   CO|           840.0|                 38163.0|
|2021-03-07|   CT|             0.0|                     0.0|
|2021-03-07|   DC|           146.0|                  5726.0|
|2021-03-07|   DE|           215.0|                  5867.0|
+----------+-----+--------

In [4]:
def ConvertDateStringToDate (glueContext, dfc) -> DynamicFrameCollection:
    sparkDF = dfc.select(list(dfc.keys())[0]).toDF()
    sparkDF.createOrReplaceTempView("inputTable")

    df = spark.sql("select TO_DATE(CAST(UNIX_TIMESTAMP(date, 'yyyyMMdd') AS TIMESTAMP)) as date, \
                           state , \
                           positiveIncrease ,  \
                           totalTestResultsIncrease \
                    from   inputTable")

    dyf = DynamicFrame.fromDF(df, glueContext, "results")
    return DynamicFrameCollection({"CustomTransform0": dyf}, glueContext)

Transform = ConvertDateStringToDate(glueContext, DynamicFrameCollection({"DataSource0": DataSource0}, glueContext))
resultDF = Transform.select(list(Transform.keys())[0]).toDF()
resultDF.printSchema()
resultDF.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- positiveIncrease: double (nullable = true)
 |-- totalTestResultsIncrease: double (nullable = true)

+----------+-----+----------------+------------------------+
|      date|state|positiveIncrease|totalTestResultsIncrease|
+----------+-----+----------------+------------------------+
|2021-03-07|   AK|             0.0|                     0.0|
|2021-03-07|   AL|           408.0|                  2347.0|
|2021-03-07|   AR|           165.0|                  3380.0|
|2021-03-07|   AS|             0.0|                     0.0|
|2021-03-07|   AZ|          1335.0|                 45110.0|
|2021-03-07|   CA|          3816.0|                133186.0|
|2021-03-07|   CO|           840.0|                 38163.0|
|2021-03-07|   CT|             0.0|                     0.0|
|2021-03-07|   DC|           146.0|                  5726.0|
|2021-03-07|   DE|           215.0|                  5867.0|
+----------+-----+--------

In [10]:
def FilterAndCalculatePercentage (glueContext, dfc) -> DynamicFrameCollection:
    sparkDF = dfc.select(list(dfc.keys())[0]).toDF()
    sparkDF.createOrReplaceTempView("inputTable")

    df = spark.sql("select  date , \
                            state , \
                            (positiveIncrease * 100 / totalTestResultsIncrease) as positivePercentage \
                    from inputTable \
                    where state in ('NY', 'CA')")

    dyf = DynamicFrame.fromDF(df, glueContext, "results")
    return DynamicFrameCollection({"CustomTransform0": dyf}, glueContext)

TransformFilter = FilterAndCalculatePercentage(glueContext, Transform)
resultDF = TransformFilter.select(list(TransformFilter.keys())[0]).toDF()
resultDF.printSchema()
resultDF.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- positivePercentage: double (nullable = true)

+----------+-----+------------------+
|      date|state|positivePercentage|
+----------+-----+------------------+
|2021-03-07|   CA|2.8651660084393256|
|2021-03-07|   NY|  2.98066453584349|
|2021-03-06|   CA| 2.039161800068705|
|2021-03-06|   NY| 2.799745178155617|
|2021-03-05|   CA|  3.17331662330232|
|2021-03-05|   NY|3.0161483152878574|
|2021-03-04|   CA| 2.928859800897716|
|2021-03-04|   NY|2.8112955359159386|
|2021-03-03|   CA| 2.561555273655413|
|2021-03-03|   NY|  3.53282676584017|
+----------+-----+------------------+
only showing top 10 rows

In [11]:
def PivotValue (glueContext, dfc) -> DynamicFrameCollection:
    sparkDF = dfc.select(list(dfc.keys())[0]).toDF()
    sparkDF.createOrReplaceTempView("inputTable")

    df = spark.sql("select * from inputTable \
                    pivot (avg(positivePercentage) as positivePercentage \
                    for state in ('NY' as positivePercentageNY, 'CA' as positivePercentageCA))")

    dyf = DynamicFrame.fromDF(df, glueContext, "results")
    return DynamicFrameCollection({"CustomTransform0": dyf}, glueContext)

PivotValueTransform = PivotValue(glueContext, TransformFilter)
resultDF = PivotValueTransform.select(list(PivotValueTransform.keys())[0]).toDF()
resultDF.printSchema()
resultDF.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: date (nullable = true)
 |-- positivePercentageNY: double (nullable = true)
 |-- positivePercentageCA: double (nullable = true)

+----------+--------------------+--------------------+
|      date|positivePercentageNY|positivePercentageCA|
+----------+--------------------+--------------------+
|2020-08-24|  0.6577356483048798|    4.43762561010623|
|2021-01-27|   5.441599518407587|  6.9206328192228765|
|2020-08-05|  0.8752132988385534|    5.10671540308814|
|2020-07-24|  0.9842236658083574|    7.06393742912802|
|2020-11-29|   4.273455377574371|   6.663565481245652|
|2020-08-28|  0.6501339112301433|  5.7784476589100215|
|2020-08-29|  0.6764458363959818|   5.031668905882234|
|2020-04-30|  16.625821346119693|  10.888368321470402|
|2020-10-04|  1.1075963708544445|   2.659867781089102|
|2020-12-18|   5.091324658660304|  14.406907671716976|
+----------+--------------------+--------------------+
only showing top 10 rows